# 신경망

## 텐서플로 2.0의 단순 신경망을 은닉층으로 개선

계층을 추가하면 매개변수가 추가돼 모델이 더 복잡한 패턴을 기억할 수 있게 됨.

* 추가 계층은 입력 또는 출력과 직접 연결되지 않기 때문에 숨겨진 것으로 간주됨. 첫 번째 은닉층 뒤에 N_HIDDEN뉴런을 가진 두 번째 은닉층이 있고 그 다음으로 10개의 뉴런을 가진 출력 계층이 있음.

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2

mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

RESHAPED = 784
X_train = X_train.reshape(60000 ,RESHAPED).astype('float32')
X_test = X_test.reshape(10000 ,RESHAPED).astype('float32')

X_train, X_test = X_train / 255.0 , X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_train.shape[0], 'test samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                            input_shape=(RESHAPED,),
                            name='dense_layer', activation='relu'))
model.add(keras.layers.Dense(N_HIDDEN,
                            name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dense(NB_CLASSES,
                            name='dense_layer_3', activation='softmax'))

model.summary()

model.compile(optimizer='SGD',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train,
         batch_size=BATCH_SIZE, epochs=EPOCHS,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f'Test accuracy: {test_acc}')

60000 train samples
60000 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 1s 3ms/step - loss: 1.3748 - accuracy: 0.6764 - val_loss: 0.6958 - val_accuracy: 0.8512
Epoch 2/50
375/375 [==============================] - 1s 2ms/step - loss: 0.5725 - accuracy: 0.8558 - val_loss: 0.4432 - val_accuracy: 0.8848
Epoch 3/50
375/375 [==============================] - 1s 

## 텐서플로에서 드롭아웃으로 단순망 개선

무작위 드롭아웃으로 신경망의 일반화를 향상시키는데 도움이 되는 유용한 중복 패턴을 학습시킨다는 것.

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

RESHAPED = 784
X_train = X_train.reshape(60000 ,RESHAPED).astype('float32')
X_test = X_test.reshape(10000 ,RESHAPED).astype('float32')

X_train, X_test = X_train / 255.0 , X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_train.shape[0], 'test samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                            input_shape=(RESHAPED,),
                            name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
                            name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
                            name='dense_layer_3', activation='softmax'))

model.summary()

model.compile(optimizer='SGD',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train,
         batch_size=BATCH_SIZE, epochs=EPOCHS,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f'Test accuracy: {test_acc}')

60000 train samples
60000 test samples
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 1s 3ms/step - loss: 1

313/313 [==============================] - 1s 2ms/step - loss: 0.1267 - accuracy: 0.9617
Test accuracy: 0.9617000222206116


훈련 정확도는 테스트 정확도보다는 높아야 한다. 그렇지 않다면 충분히 오랫동안 훈련을 받지 못한 것일 수 있음.
- 에폭의 수를 좀 더 늘려야 함. 그러나 바로 에폭을 늘리기 전에 훈련이 더 빨리 수렴될 수 있또록 몇 가지 다른 개념을 도입할 필요가 있음.



## 텐서플로 2.0에서 여러 최적화기 테스트

도랑찾기.

In [6]:
# RMSProp optimizer

import tensorflow as tf
import numpy as np
from tensorflow import keras

EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

RESHAPED = 784
X_train = X_train.reshape(60000 ,RESHAPED).astype('float32')
X_test = X_test.reshape(10000 ,RESHAPED).astype('float32')

X_train, X_test = X_train / 255.0 , X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_train.shape[0], 'test samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                            input_shape=(RESHAPED,),
                            name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
                            name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
                            name='dense_layer_3', activation='softmax'))

model.summary()

model.compile(optimizer='RMSProp',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train,
         batch_size=BATCH_SIZE, epochs=EPOCHS,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f'Test accuracy: {test_acc}')

60000 train samples
60000 test samples
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 1s 3ms/step - loss: 0

313/313 [==============================] - 1s 2ms/step - loss: 0.1391 - accuracy: 0.9784
Test accuracy: 0.9783999919891357


In [7]:
#  Adam optimizer

import tensorflow as tf
import numpy as np
from tensorflow import keras

EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

RESHAPED = 784
X_train = X_train.reshape(60000 ,RESHAPED).astype('float32')
X_test = X_test.reshape(10000 ,RESHAPED).astype('float32')

X_train, X_test = X_train / 255.0 , X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_train.shape[0], 'test samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                            input_shape=(RESHAPED,),
                            name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
                            name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
                            name='dense_layer_3', activation='softmax'))

model.summary()

model.compile(optimizer='Adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train,
         batch_size=BATCH_SIZE, epochs=EPOCHS,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f'Test accuracy: {test_acc}')

60000 train samples
60000 test samples
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 1s 3ms/step - loss: 0

313/313 [==============================] - 1s 2ms/step - loss: 0.0808 - accuracy: 0.9806
Test accuracy: 0.9805999994277954


## 에폭 수 증가시키기

## 최적화기 학습률 조절

## 내부 은닉층 개수 증가

## 배치 계산 크기 증가

# 정규화

## 과적합을 피하기 위한 정규화 적용

복잡도의 증가는 두 가지 부정적인 결과를 초래할 수 있음.

1. 복잡한 모델을 실행하려면 상당한 시간이 소요될 수 있음.


2. 복잡한 모델은 훈련 데이터에서는 매우 우수한 성과를 달성할 수 있지만 검증 데이터에서는 상당히 나쁜 성과를 거둘 수 있음.

머신러닝 모델 정규화 3가지

- L1 정규화 ; 절댓값

- L2 정규화 ; 제곱

- 일래스틱 정규화 ; 앞 선 두 기법의 조합 

## 배치 정규화의 이해

배치 정규화는 또 다른 형태의 정규화.

훈련 과정에서 초기 계층의 가중치는 변경될 것이 당연하므로 이후 계층의 입력은 크게 변경될 수 있음.

즉 , 각 계층은 모든 배치마다 가중치를 지속적으로 다른 분포로 다시 조정해야 함.

이로 인해 모델의 훈련 속도가 크게 떨어질 수 있음.

__핵심 아이디어는 각 배치와 각 에폭에 대해 계층 입력이 좀 더 유사한 분포를 갖게 하자는 것임.__

또 다른 문제는 시그모이드 활성화 함수는 0 근처에서 아주 잘 작동하나 값이 0 에서 상당히 멀어지면 ' 고착 ' 되는 경향이 있음. ; 해당 뉴런은 자신의 가중치를 갱신할 수 없게 됨.

-->계층 출력을 0에 가까운 가우시안 분포 단위로 변환 하는 것.

# 감정 분석

In [10]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing
import tensorflow_datasets as tfds

max_len = 200
n_words = 10000
dim_embedding = 256
EPOCHS = 20
BATCH_SIZE = 500

def load_data():
    (X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=n_words)
    X_train = preprocessing.sequence.pad_sequences(X_train,maxlen=max_len)
    X_test = preprocessing.sequence.pad_sequences(X_test,maxlen=max_len)
    return (X_train, y_train), (X_test, y_test)

In [11]:
def build_model():
    model = models.Sequential()
    # 입력 ; embedding layer
    # 모델의 크기의 정수 행렬을 입력으로 취함 (batch , input_length)
    # 모델의 출력은 차원 ( input_length , dim_embedding )
    # 입력 중 가장 큰 정수는 n_words보다 작거나 같음.
    model.add(layers.Embedding(n_words, dim_embedding, input_length=max_len))
    model.add(layers.Dropout(0.3))
    # 각 n_words 특징에서 특징 벡터의 최댓값을 취함.
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [12]:
(X_train, y_train), (X_test, y_test) = load_data()
model = build_model()
model.summary()

model.compile(optimizer = 'adam',
             loss='binary_crossentropy',
             metrics = ['accuracy'])

score = model.fit(X_train, y_train,
                  epochs = EPOCHS,
                  batch_size = BATCH_SIZE,
                  validation_data = (X_test, y_test))

score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f'\nTest score: {score[0]}')
print(f'Test accuracy: {score[1]}')

17465344/17464789 [==============================] - 2s 0us/step
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 256)          2560000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 200, 256)          0         
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,593,025
Trainable params: 2,593,025
Non

# 출력 예측

- model.evaluate() 는 손실 값을 계산하는데 사용함.



- model.predict_class() 는 범주 출력을 계산



- model.predict_proba() 부류 확률을 계산하는 데 사용함.

# 역전파에 대한 실용적 개괄

다층 퍼셉트론은 역전파라는 프로세스를 통해 훈련 데이터에서 학습함.

--> 오류가 발견되는 즉시 점진적으로 수정해 나가는 방법.

처음에는 모든 가중치에 __임의의 값__을 할당함.

그런 다음 훈련 집합의 각 입력에 대해 신경망이 활성화됨.

값은 입력 단계에서 은닉 단계를 통해 출력 단계로 순전파되어 예측이 이뤄짐. 

훈련 집합에서 실제 관측값을 알기 때문에 예측에서 발생한 오차를 계산할 수 있음.

__역전파의 핵심 아이디어__

그레디언트 하강과 같은 적절한 최적화 알고리즘을 사용해 오차를 줄이려는 목적으로 신경망 가중치를 조정하고자 오차를 역으로 전파하는 것임.

__신경망에서 실제로 중요한 것은 단일 뉴런의 출력이 아니라 각 계층에서 조정된 종합적인 가중치임.__

